In [308]:
!pip install datasets
!pip install torch
!pip install transformers
!pip install evaluate


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [309]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# Extracting Data From PreProcessing Models

In [310]:
import pandas as pd
data = pd.read_csv('try_cleaned_data.csv')

In [311]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.util import ngrams
import numpy as np

#Split data into train,val,test in 0.6:0.2:0.2 ratio 
X_train, X_test, y_train, y_test = train_test_split(data["title"], data["labels"], test_size=0.2, shuffle=True, random_state=4222)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, shuffle=True, random_state=4222)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)


(8214,)
(2738,)
(2738,)


## Attempt 1 : Changing Train, Test and Val into TFIDF matrix for input 

In [312]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


def tfidf_embedding(X_train,X_val, X_test,y_train, y_val,y_test):
    vectorizer = CountVectorizer().fit(X_train.values.astype('U'))
    X_train_count = vectorizer.transform(X_train.values.astype('U'))
    X_val_count = vectorizer.transform(X_val.values.astype('U'))
    X_test_count = vectorizer.transform(X_test.values.astype('U'))

    transformer = TfidfTransformer().fit(X_train_count)
    X_train_feature = transformer.transform(X_train_count)
    X_val_feature = transformer.transform(X_val_count)
    X_test_feature = transformer.transform(X_test_count)

    Xtrainmatrix = X_train_feature.todense()
    Xvalmatrix = X_val_feature.todense()
    Xtestmatrix = X_test_feature.todense()
    print(Xtrainmatrix.shape)
    print(Xvalmatrix.shape)
    print(Xtestmatrix.shape)
    return Xtrainmatrix,Xvalmatrix,Xtestmatrix

    

## Attempt 2: NGRAM

In [313]:
def NGRAM_Embedding(X_train,X_val, X_test,y_train, y_val,y_test):
  train_data_title= X_train.values.astype('U').tolist()
  train_data_labels = y_train
  def extract_ngrams(text, num):
      words = nltk.word_tokenize(text)
      n_grams = ngrams(words, num)
      return list(n_grams)

  bigrams_dataset = set()
  trigrams_dataset = set()
  for i in (X_train.values.astype('U')):
    text = i
    bigrams = extract_ngrams(text, 2)
    trigrams = extract_ngrams(text, 3)
    for bigram in bigrams:
      bigrams_dataset.add(bigram)
    for trigram in trigrams:
      trigrams_dataset.add(trigram)

  ngram_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
  ngram_matrix = ngram_vectorizer.fit_transform(train_data_title)
  ngram_dense_matrix = ngram_matrix.todense()
  feature_names = ngram_vectorizer.get_feature_names_out()

  ngram_df = pd.DataFrame(ngram_matrix.toarray(), columns=feature_names)
  clf = RandomForestClassifier()
  clf.fit(ngram_matrix, train_data_labels)  # 'labels' is the array of sentiment labels
  importances = clf.feature_importances_
  sorted_indices = np.argsort(importances)[::-1]
  # Select the top N feature indices
  N = 1000  # For example, select the top 1000 features
  top_feature_indices = sorted_indices[:N]
  # Select the top N features from the original feature matrix
  X_top_ngrams = ngram_matrix[:, top_feature_indices]

  ngram_matrix_train = ngram_vectorizer.fit_transform(train_data_title)
  ngram_matrix_val = ngram_vectorizer.transform(X_val.values.astype('U').tolist())
  ngram_matrix_test = ngram_vectorizer.transform(X_test.values.astype('U').tolist())

  Xtrainmatrix = ngram_matrix_train[:, top_feature_indices].todense()
  Xvalmatrix = ngram_matrix_val[:, top_feature_indices].todense()
  Xtestmatrix = ngram_matrix_test[:, top_feature_indices].todense()
  print(Xtrainmatrix.shape)
  print(Xvalmatrix.shape)
  print(Xtestmatrix.shape)

  return Xtrainmatrix,Xvalmatrix,Xtestmatrix


# Preparing inputs 
adjust input value to determine embedding type

In [314]:
def acquireinputs(Methods, X_train,X_val, X_test,y_train, y_val,y_test): 
    if Methods == "TFIDF":
        Xtrainmatrix,Xvalmatrix,Xtestmatrix = tfidf_embedding(X_train,X_val, X_test,y_train, y_val,y_test)
    if Methods == "NGRAM":
        Xtrainmatrix,Xvalmatrix,Xtestmatrix = NGRAM_Embedding(X_train,X_val, X_test,y_train, y_val,y_test)
    train_vectors = Xtrainmatrix
    val_vectors = Xvalmatrix
    test_vectors = Xtestmatrix

    train_labels = y_train
    val_labels = y_val
    test_labels = y_test

    if Methods == "TFIDF":
        vecsize = Xtrainmatrix.shape[1]
    if Methods == "NGRAM":
        vecsize = 1000
    train_vectors1 = torch.Tensor(train_vectors).reshape(len(train_vectors),1,vecsize)
    val_vectors1 = torch.Tensor(val_vectors).reshape(len(val_vectors),1,vecsize)
    test_vectors1 = torch.Tensor(test_vectors).reshape(len(test_vectors),1,vecsize)

    train_labels1 = torch.Tensor(train_labels.values)
    val_labels1 = torch.Tensor(val_labels.values)
    test_labels1 = torch.Tensor(test_labels.values)

    print(f"Utilising {Methods}: ")

    print(train_vectors1.shape)
    print(train_labels1.shape)

    print(val_vectors1.shape)
    print(val_labels1.shape)

    print(test_vectors1.shape)
    print(test_labels1.shape)

    return train_vectors1,val_vectors1,test_vectors1, train_labels1, val_labels1, test_labels1



Method = "TFIDF"
Method = "NGRAM"
train_vectors1,val_vectors1,test_vectors1, train_labels1, val_labels1, test_labels1= acquireinputs(Method, X_train,X_val, X_test,y_train, y_val,y_test)

(8214, 1000)
(2738, 1000)
(2738, 1000)
Utilising NGRAM: 
torch.Size([8214, 1, 1000])
torch.Size([8214])
torch.Size([2738, 1, 1000])
torch.Size([2738])
torch.Size([2738, 1, 1000])
torch.Size([2738])


# Model 
Model comprises of 2 Convolutional Layer, LSTM layer followed by an ATTN mechanism layer. 

In [315]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the first convolutional layer
        self.conv1 = nn.Conv1d(1, 4, 3, 1,1, bias=True)
        # Define the batch normalization layer for the first conv layer
        self.Bn1 = nn.BatchNorm1d(4)
        # Define the max pooling layer for the first conv layer
        self.pool1=nn.MaxPool1d(kernel_size=3, stride=3)

        self.conv2 = nn.Conv1d(1, 4, 3, 1,1, bias=True)
        self.Bn2 = nn.BatchNorm1d(4)
        self.pool2=nn.MaxPool1d(kernel_size=3, stride=3)

        #39536 for TFIDF 
        if Method =="TFIDF":
            inp = 34816
        if Method =="NGRAM":
            inp = 2664
        self.bi_lstm1 = nn.LSTM(input_size=inp, hidden_size=100, num_layers=1, batch_first=True, bidirectional=False) 
        self.fc1 = nn.Linear(100, 100, bias=True)

        self.self_attn_1 = nn.MultiheadAttention(embed_dim=100, num_heads=4)
        self.fc2 = nn.Linear(100, 3, bias=True)

    def forward(self, x):
        # Pass input through the first convolutional layer, then through the ReLU activation function, then through max pooling
        x_layer1 = self.pool1(F.relu(self.Bn1(self.conv1(x))))
        x_layer2 = self.pool1(F.relu(self.Bn2(self.conv2(x))))
        # Concatenate the outputs of the three layers along the channel dimension
        x = torch.cat((x_layer1, x_layer2), 1)
        # Flatten the tensor for the fully connected layers
        x = torch.flatten(x, 1)
        # Pass input through the LSTM layer
        x, _ = self.bi_lstm1(x)
        # Pass output of LSTM layer through the first fully connected layer, then through the ReLU activation function
        x = F.relu(self.fc1(x))
        # Reshape the tensor for the self-attention layer
        x = x.view(-1, 1, 100)
        # Pass tensor through the self-attention layer
        x, _ = self.self_attn_1(x.permute(1, 0, 2), x.permute(1, 0, 2), x.permute(1, 0, 2))
        # Reshape tensor back to original shape
        x = x.permute(1, 0, 2)
        x = x.view(-1, 100)
        # Pass the output through the final fully connected layer for classification
        x = self.fc2(x)
        return x



In [316]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()  # Set the model to training mode

    for batch_idx, (data, target) in enumerate(train_loader):  # Loop over each batch from the training set
        data, target = data.to(device), target.to(device)  # Move the data to the device that is used
        target = target  
        target = target.long()  # Make sure that target data is long type (necessary for loss function)
        optimizer.zero_grad()  # Clear gradients from the previous training step
        output = model(data)  # Run forward pass (model predictions)
        loss = F.cross_entropy(output, target)  # Calculate the loss between the output and target
        loss.backward()  # Perform backpropagation (calculate gradients of loss w.r.t. parameters)
        optimizer.step()  # Update the model parameters
        if batch_idx % args.log_interval == 10:  # Print log info for specified interval
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0

    with torch.no_grad():  # Deactivates autograd, reduces memory usage and speeds up computations
        for data, target in test_loader:  # Loop over each batch from the testing set
            data, target = data.to(device), target.to(device)  # Move the data to the device that is used
            target = target  # Adjust the target values
            output = model(data)  # Run forward pass (model predictions)
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability as the predicted output
            correct += pred.eq(target.view_as(pred)).sum().item()  # Count correct predictions

    test_loss /= len(test_loader.dataset)  # Calculate the average loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
    return correct  # Return the number of correctly classified samples


In [317]:
class Args:
  epochs = 10
  lr = 1.0
  use_cuda=False
  gamma = 0.7
  log_interval = 10
  no_cuda = False
  seed = 1

args = Args()

In [318]:
torch.manual_seed(args.seed)
device = torch.device("cuda" if args.use_cuda else "cpu")
model = Net().to(device)
for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())
#Form training and testing dataset
optimizer = optim.Adadelta(model.parameters(), lr=1)
train_dataset = torch.utils.data.TensorDataset(train_vectors1, train_labels1)
test_dataset = torch.utils.data.TensorDataset(test_vectors1, test_labels1)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

#Model training
print(f"{Method}")
ACC = 0
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    ACC_ = test(model, device, test_loader)
    if ACC_>ACC or ACC_ == ACC:
        ACC = ACC_
        if Method == "TFIDF":
            torch.save(model.state_dict(), "tfidfcnn_lstm_att.pt")
        if Method == "NGRAM":
            torch.save(model.state_dict(), "ngramcnn_lstm_att.pt")
    scheduler.step()
print(ACC)


conv1.weight 	 torch.Size([4, 1, 3])
conv1.bias 	 torch.Size([4])
Bn1.weight 	 torch.Size([4])
Bn1.bias 	 torch.Size([4])
Bn1.running_mean 	 torch.Size([4])
Bn1.running_var 	 torch.Size([4])
Bn1.num_batches_tracked 	 torch.Size([])
conv2.weight 	 torch.Size([4, 1, 3])
conv2.bias 	 torch.Size([4])
Bn2.weight 	 torch.Size([4])
Bn2.bias 	 torch.Size([4])
Bn2.running_mean 	 torch.Size([4])
Bn2.running_var 	 torch.Size([4])
Bn2.num_batches_tracked 	 torch.Size([])
bi_lstm1.weight_ih_l0 	 torch.Size([400, 2664])
bi_lstm1.weight_hh_l0 	 torch.Size([400, 100])
bi_lstm1.bias_ih_l0 	 torch.Size([400])
bi_lstm1.bias_hh_l0 	 torch.Size([400])
fc1.weight 	 torch.Size([100, 100])
fc1.bias 	 torch.Size([100])
self_attn_1.in_proj_weight 	 torch.Size([300, 100])
self_attn_1.in_proj_bias 	 torch.Size([300])
self_attn_1.out_proj.weight 	 torch.Size([100, 100])
self_attn_1.out_proj.bias 	 torch.Size([100])
fc2.weight 	 torch.Size([3, 100])
fc2.bias 	 torch.Size([3])
NGRAM

Test set: Average loss: 0.0000, 

# Test

In [319]:
model1 = Net().to(device)
if Method == "NGRAM":
    model1.load_state_dict(torch.load('ngramcnn_lstm_att.pt'))
if Method == "TFIDF":
    model1.load_state_dict(torch.load('tfidfcnn_lstm_att.pt'))
print(f"Method {Method}")
finaltest = torch.utils.data.TensorDataset(test_vectors1, test_labels1)
finalloader = torch.utils.data.DataLoader(finaltest, batch_size=8, shuffle=True)
ACC_ = test(model1, device, finalloader)
print(ACC_)

Method NGRAM

Test set: Average loss: 0.0000, Accuracy: 2037/2738 (74%)

2037


In [320]:
Method = "TFIDF"
model1 = Net().to(device)
train_vectors1,val_vectors1,test_vectors2, train_labels1, val_labels1, test_labels2= acquireinputs(Method, X_train,X_val, X_test,y_train, y_val,y_test)
if Method == "NGRAM":
    model1.load_state_dict(torch.load('ngramcnn_lstm_att.pt'))
if Method == "TFIDF":
    model1.load_state_dict(torch.load('tfidfcnn_lstm_att.pt'))
print(f"Method {Method}")
finaltest = torch.utils.data.TensorDataset(test_vectors2, test_labels2)
finalloader = torch.utils.data.DataLoader(finaltest, batch_size=8, shuffle=True)
ACC_ = test(model1, device, finalloader)
print(ACC_)

(8214, 13056)
(2738, 13056)
(2738, 13056)
Utilising TFIDF: 
torch.Size([8214, 1, 13056])
torch.Size([8214])
torch.Size([2738, 1, 13056])
torch.Size([2738])
torch.Size([2738, 1, 13056])
torch.Size([2738])
Method TFIDF

Test set: Average loss: 0.0000, Accuracy: 1945/2738 (71%)

1945


# Conclusion

combination of LSTM CNN attn works well to extract meaning behind text via attn mechnaism. Works better with NGRAM compared to TFIDF

# Other attempts at improving our model (not part of report)

## Word2Vec
Pre trained model does not work well with OOB due to financial jargon,
Low accuracy and did not yield strong results especially when it came to OOB words

In [321]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip


In [322]:
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# import numpy as np

In [323]:
# embeddings_dict = {}
# with open('glove.6B.50d.txt','rb') as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         vector = np.asarray(values[1:], "float32")
#         embeddings_dict[word] = vector
# embeddings_dict[b'test']

# sents = ['This is BT4222']
# MAX_NUM_WORDS = 100
# MAX_SEQUENCE_LENGTH = 20
# tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
# tokenizer.fit_on_texts(sents)
# sequences = tokenizer.texts_to_sequences(sents)

# word_index = tokenizer.word_index
# data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# from keras.layers import Embedding
# from keras.initializers import Constant
# embeddings_dict={}
# EMBEDDING_DIM = embeddings_dict.get(b'a').shape[0]
# num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
# embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
# for word, i in word_index.items():
#     if i > MAX_NUM_WORDS:
#         continue
#     embedding_vector = embeddings_dict.get(word.encode("utf-8"))
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

## Fast Text
To overcome OOB problems, however, weaker accuracy compared to NGRAM

In [324]:
# import pandas as pd
# from nltk.tokenize import word_tokenize
# from gensim.models import FastText
# import numpy as np

# from sklearn.model_selection import train_test_split
# train_data, test_data = train_test_split(df, test_size=0.2)

# model = FastText(sentences=train_data['title'], vector_size =300, window=7, min_count=3, epochs = 5, seed =42, sg =1 )
# wv = model.wv
# def get_sentence_vectors(sentences):
#     vectors = []
#     for sentence in sentences:
#         sentence_vectors = [wv[word] for word in sentence if word in wv]
#         vectors.append((np.mean(sentence_vectors, axis=0)))
#     return vectors

# train_vectors = get_sentence_vectors(train_data['title'])
# print("vector of the first training sentence:", train_vectors[0])
# test_vectors = get_sentence_vectors(test_data['title'])

In [325]:
# from transformers import BertTokenizer
# import torch

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# token_lens = []
# for txt in dataset['train']['summary_detail_with_title']:
#   tokens = tokenizer.encode(txt, max_length=512)
#   token_lens.append(len(tokens))

# from transformers import BertModel
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertModel.from_pretrained("bert-base-uncased")
# # get the embedding vector for the word "example"
# example_token_id = tokenizer.convert_tokens_to_ids(["example"])[0]
# example_embedding = model.embeddings.word_embeddings(torch.tensor([example_token_id]))
# example_embedding


In [326]:
# class GPReviewDataset(Dataset):

#   def __init__(self, reviews, targets, tokenizer, max_len):
#     self.reviews = reviews
#     self.targets = targets
#     self.tokenizer = tokenizer
#     self.max_len = max_len

#   def __len__(self):
#     return len(self.reviews)

#   def __getitem__(self, item):
#     review = str(self.reviews[item])
#     target = self.targets[item]

#     encoding = self.tokenizer.encode_plus(
#       review,
#       add_special_tokens=True,
#       max_length=self.max_len,
#       return_token_type_ids=False,
#       pad_to_max_length=True,
#       return_attention_mask=True,
#       return_tensors='pt',
#     )

#     return {
#       'review_text': review,
#       'input_ids': encoding['input_ids'].flatten(),
#       'attention_mask': encoding['attention_mask'].flatten(),
#       'targets': torch.tensor(target, dtype=torch.long)
#     }

In [327]:
# # Load the BERT tokenizer from the "bert-base-uncased" pre-trained model
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(modelname)
# def tokenize_function(examples):
# # Tokenize the "text" column of the examples, adding padding to the maximum length and truncating if necessary
#         return tokenizer(examples["summary_detail_with_title"], padding="max_length", truncation=True)
# # Apply the tokenize function to the dataset in batches
# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# tokenized_datasets = tokenized_datasets.remove_columns(["summary_detail_with_title"])
# # Rename the "label" column to "labels" to match the expected format for training
# tokenized_datasets.set_format("torch")
# # Shuffle the dataset with a fixed seed and select a range of examples
# train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))


## Roberta

In [328]:
# # ROBERTA FIN BERT model
# #Their own model
# from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline
# model = AutoModelForSequenceClassification.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")
# tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")

# classifier=pipeline("text-classification",model=model, tokenizer=tokenizer)
# output=classifier(dataset['train']['summary_detail_with_title'][0])
# print(output)

## FinBERT

In [329]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-classification", model="ProsusAI/finbert")
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
# model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
# # output=classifier(dataset['train']['summary_detail_with_title'][0])
# # print(output)

## BERT/ Distilbert

In [330]:
# modelname = "bert-base-uncased"
# modelname = "distilbert-base-uncased"
# modelname = "distilbert-base-uncased"


# # modelname = "distilbert-base-uncased"
# # id2label = {0: "NEGATIVE", 1: "NEUTRAL", 2:"POSITIVE"}
# # label2id = {"NEGATIVE": 0, "NEUTRAL":1 , "POSITIVE": 2}
# # model = AutoModelForSequenceClassification.from_pretrained(
# #     "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
# # )

In [331]:
# # Load the BERT tokenizer from the "bert-base-uncased" pre-trained model
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(modelname)
# def tokenize_function(examples):
# # Tokenize the "text" column of the examples, adding padding to the maximum length and truncating if necessary
#         return tokenizer(examples["summary_detail_with_title"], padding="max_length", truncation=True)
# # Apply the tokenize function to the dataset in batches
# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# tokenized_datasets = tokenized_datasets.remove_columns(["summary_detail_with_title"])
# # Rename the "label" column to "labels" to match the expected format for training
# tokenized_datasets.set_format("torch")
# # Shuffle the dataset with a fixed seed and select a range of examples
# train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

In [332]:
# train_dataset = train_dataset.remove_columns(['summary_detail', 'title', 'topic',  '__index_level_0__',])
# eval_dataset = eval_dataset.remove_columns(['summary_detail', 'title', 'topic',  '__index_level_0__',])

In [333]:
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,TensorDataset
# batchsize=4
# # Create Dataloader
# train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batchsize)
# eval_dataloader = DataLoader(eval_dataset, batch_size=batchsize)

In [334]:
# from transformers import AutoModelForMaskedLM
# # Because we initialized BertForMaskedLM and concat is with our classifier instead of directly using BertForSequenceClassification
# # Some weights of the model checkpoint at bert-base-uncased were not used is within the expectation.
# import torch
# from torch import nn
# from torch.nn import CrossEntropyLoss
# from torch.optim import AdamW


# class Model(nn.Module):
#     def __init__(self,output_dim,dropout_rate):
#         super(Model,self).__init__()
#         self.encoder=AutoModelForMaskedLM.from_pretrained(modelname, output_hidden_states=True, return_dict=True)
#         self.dropout=nn.Dropout(dropout_rate)
#         # For the "bert-base-uncased" model, each hidden state has a dimension of 768.
#         # the value 3072=4*768 corresponds to the total dimension of the concatenated hidden states from the BERT model.
#         self.classifier=nn.Linear(3072,output_dim)


#     def forward(self,input_ids,token_type_ids,attention_mask):
#         outputs = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
#         hidden_states = torch.cat(tuple([outputs.hidden_states[i] for i in [-1, -2, -3, -4]]), dim=-1) # [bs, seq_len, hidden_dim*4]
#         # We are actually extracting the hidden state of the [CLS] token for each sequence in the batch.
#         # This [CLS] token's hidden state is typically used as a fixed-size representation of the entire sequence.
#         # This representation has been learned during BERT's pretraining to capture important information for various tasks.
#         # In the context of classification, you can think of the [CLS] token's hidden state as a summary of the sequence's content,
#         # which is then fed into the linear classifier to make predictions for the task at hand.
#         x=self.dropout(hidden_states[:, 0, :])
#         x=self.classifier(x)
#         return x

# model = Model(output_dim=3, dropout_rate = 0.5)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# loss_fct = CrossEntropyLoss()
# optimizer = AdamW(model.parameters(), lr=5e-5)

In [335]:
# from transformers import get_scheduler
# from tqdm.auto import tqdm
# import evaluate

# epochs = 2
# num_training_steps = epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
# name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
#     )
# metric = evaluate.load("accuracy")
# progress_bar = tqdm(range(num_training_steps))


# for epoch in range(epochs):
#         for batch in train_dataloader:
#             model.train()
#             # Loop through batches in the training data loader
#             label_ids = batch['labels']
#             input_ids = batch['input_ids']
#             token_type_ids = None
#             # When using BERT for tasks like single-text classification or sequence labeling, the token_type_ids is an optional parameter, commonly set to None.
#             attention_mask = batch['attention_mask']
#             # Perform a forward pass through the model to get logits
#             logits = model(input_ids, token_type_ids, attention_mask)

#             # Calculate the loss using the provided loss function
#             loss = loss_fct(logits, label_ids.view(-1))
#             # Perform backward pass and update model parameters
#             loss.backward()
#             optimizer.step()
#             lr_scheduler.step()
#             optimizer.zero_grad() # Clear accumulated gradients
#             progress_bar.update(1) # Update progress bar

#         # Set the model to evaluation mode for validation
#         model.eval()
#         for batch in eval_dataloader:
#             batch = {k: v.to(device) for k, v in batch.items()}
#             with torch.no_grad(): # disable gradient computation
#                 label_ids = batch['labels']
#                 input_ids = batch['input_ids']
#                 token_type_ids = None
#                 attention_mask = batch['attention_mask']
#                 logits = model(input_ids, token_type_ids, attention_mask)
#                 loss = loss_fct(logits, label_ids.view(-1))

#             # Get predicted labels by selecting the class with the highest probability
#             predictions = torch.argmax(logits, dim=-1)
#             metric.add_batch(predictions=predictions, references=batch["labels"])

#         acc = metric.compute()
#         print(f'Epoch {epoch+1}')
#         print(f'val_loss : {loss}')
#         print(f"val_accuracy: {acc['accuracy'] * 100}")
#         print(25*'==')

In [336]:
# # D
# #Their own model
# from transformers import DistilBertTokenizer, DistilBertModel
# from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer,DataCollatorWithPadding
# import numpy as np
# import evaluate

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# id2label = {0: "NEGATIVE", 1: "NEUTRAL", 2:"POSITIVE"}
# label2id = {"NEGATIVE": 0, "NEUTRAL":1 , "POSITIVE": 2}


# model = AutoModelForSequenceClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
# )

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# accuracy = evaluate.load("accuracy")
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)

# training_args = TrainingArguments(
#     output_dir="my_awesome_model",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=True,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["test"],
#     tokenizer=tokenizer,
#     data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
#     compute_metrics=compute_metrics,
# )

# trainer.train()

In [337]:
# classifier=pipeline("text-classification",model=model, tokenizer=tokenizer)
# output=classifier(dataset['train']['summary_detail_with_title'][0])
# print(output)

## HANN model from Hierarchical Attentional Hybrid Neural Networks for Document Classification 2019

In [338]:
# class HAHNetwork():
#     def __init__(self):
#         self.model = None
#         self.MAX_SENTENCE_LENGTH = 0
#         self.MAX_SENTENCE_COUNT = 0
#         self.VOCABULARY_SIZE = 0
#         self.word_embedding = None
#         self.model = None
#         self.word_attention_model = None
#         self.tokenizer = None
#         self.class_count = 2

#     def build_model(self, n_classes=2, embedding_dim=200, embeddings_path=False):

#         l2_reg = regularizers.l2(0.001)

#         embedding_weights = np.random.normal(0, 1, (len(self.tokenizer.word_index) + 1, embedding_dim))

#         if embeddings_path is not None:

#             if word_embedding_type is 'from_scratch':
#                 # FastText
#                 filename = './fasttext_model.txt'
#                 model =  gensim.models.FastText.load(filename)

#                 embeddings_index = model.wv
#                 embedding_matrix = np.zeros( ( len(self.tokenizer.word_index) + 1, embedding_dim) )
#                 for word, i in self.tokenizer.word_index.items():
#                     try:
#                         embedding_vector = embeddings_index[word]
#                         if embedding_vector is not None:
#                             embedding_matrix[i] = embedding_vector
#                     except Exception as e:
#                         #print(str(e))
#                         continue


#             else:
#                 embedding_dim = 300
#                 embedding_matrix = load_subword_embedding_300d(self.tokenizer.word_index)

#             embedding_weights = embedding_matrix

#         sentence_in = Input(shape=(self.MAX_SENTENCE_LENGTH,), dtype='int32', name="input_1")

#         embedding_trainable = True



#         if word_embedding_type is 'pre_trained':
#             embedding_trainable = False

#         embedded_word_seq = Embedding(
#             self.VOCABULARY_SIZE,
#             embedding_dim,
#             weights=[embedding_weights],
#             input_length=self.MAX_SENTENCE_LENGTH,
#             trainable=embedding_trainable,
#             #mask_zero=True,
#             mask_zero=False,
#             name='word_embeddings',)(sentence_in)



#         dropout = Dropout(0.2)(embedded_word_seq)
#         filter_sizes = [3,4,5]
#         convs = []
#         for filter_size in filter_sizes:
#             conv = Conv1D(filters=64, kernel_size=filter_size, padding='same', activation='relu')(dropout)
#             pool = MaxPool1D(filter_size)(conv)
#             convs.append(pool)

#         concatenate = Concatenate(axis=1)(convs)

#         if rnn_type is 'GRU':
#             #word_encoder = Bidirectional(CuDNNGRU(50, return_sequences=True, dropout=0.2))(concatenate)
#             dropout = Dropout(0.1)(concatenate)
#             word_encoder = Bidirectional(CuDNNGRU(50, return_sequences=True))(dropout)
#         else:
#             word_encoder = Bidirectional(
#                 LSTM(50, return_sequences=True, dropout=0.2))(embedded_word_seq)


#         dense_transform_word = Dense(
#             100,
#             activation='relu',
#             name='dense_transform_word',
#             kernel_regularizer=l2_reg)(word_encoder)

#         # word attention
#         attention_weighted_sentence = Model(
#             sentence_in, Attention(name="word_attention")(dense_transform_word))

#         self.word_attention_model = attention_weighted_sentence

#         attention_weighted_sentence.summary()

#         # sentence-attention-weighted document scores

#         texts_in = Input(shape=(self.MAX_SENTENCE_COUNT, self.MAX_SENTENCE_LENGTH), dtype='int32', name="input_2")

#         attention_weighted_sentences = TimeDistributed(attention_weighted_sentence)(texts_in)


#         if rnn_type is 'GRU':
#             #sentence_encoder = Bidirectional(GRU(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.2))(attention_weighted_sentences)
#             dropout = Dropout(0.1)(attention_weighted_sentences)
#             sentence_encoder = Bidirectional(CuDNNGRU(50, return_sequences=True))(dropout)
#         else:
#             sentence_encoder = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.2))(attention_weighted_sentences)


#         dense_transform_sentence = Dense(
#             100,
#             activation='relu',
#             name='dense_transform_sentence',
#             kernel_regularizer=l2_reg)(sentence_encoder)

#         # sentence attention
#         attention_weighted_text = Attention(name="sentence_attention")(dense_transform_sentence)


#         prediction = Dense(n_classes, activation='softmax')(attention_weighted_text)

#         model = Model(texts_in, prediction)
#         model.summary()


#         optimizer=Adam(lr=learning_rate, decay=0.0001)

#         model.compile(
#                       optimizer=optimizer,
#                       loss='categorical_crossentropy',
#                       metrics=['accuracy'])

#         return model


#     def get_tokenizer_filename(self, saved_model_filename):
#         return saved_model_filename + '.tokenizer'

#     def create_reverse_word_index(self):
#         self.reverse_word_index = {value:key for key,value in self.tokenizer.word_index.items()}

#     def encode_texts(self, texts):
#         encoded_texts = np.zeros((len(texts), self.MAX_SENTENCE_COUNT, self.MAX_SENTENCE_LENGTH))
#         for i, text in enumerate(texts):
#             encoded_text = np.array(pad_sequences(
#                 self.tokenizer.texts_to_sequences(text),
#                 maxlen=self.MAX_SENTENCE_LENGTH))[:self.MAX_SENTENCE_COUNT]
#             encoded_texts[i][-len(encoded_text):] = encoded_text
#         return encoded_texts


#     def encode_input(self, x, log=False):
#         x = np.array(x)
#         if not x.shape:
#             x = np.expand_dims(x, 0)
#         texts = np.array([normalize(text) for text in x])
#         return self.encode_texts(texts)


#     def predict(self, x):
#             encoded_x = self.encode_texts(x)
#             return self.model.predict(encoded_x)


#     def activation_maps(self, text, websafe=False):
#         normalized_text = normalize(text)

#         encoded_text = self.encode_input(text)[0]

#         # get word activations

#         hidden_word_encoding_out = Model(
#             inputs=self.word_attention_model.input,
#             outputs=self.word_attention_model.get_layer('dense_transform_word').output)


#         hidden_word_encodings = hidden_word_encoding_out.predict(encoded_text)

#         word_context = self.word_attention_model.get_layer('word_attention').get_weights()[0]


#         dot = np.dot(hidden_word_encodings, word_context)

#         #u_wattention = encoded_text*np.exp(np.squeeze(dot))
#         u_wattention = encoded_text

#         if websafe:
#             u_wattention = u_wattention.astype(float)

#         nopad_encoded_text = encoded_text[-len(normalized_text):]
#         nopad_encoded_text = [list(filter(lambda x: x > 0, sentence)) for sentence in nopad_encoded_text]
#         reconstructed_texts = [[self.reverse_word_index[int(i)]
#                                 for i in sentence] for sentence in nopad_encoded_text]
#         nopad_wattention = u_wattention[-len(normalized_text):]
#         nopad_wattention = nopad_wattention/np.expand_dims(np.sum(nopad_wattention, -1), -1)
#         nopad_wattention = np.array([attention_seq[-len(sentence):]
#                             for attention_seq, sentence in zip(nopad_wattention, nopad_encoded_text)])
#         word_activation_maps = []
#         for i, text in enumerate(reconstructed_texts):
#             word_activation_maps.append(list(zip(text, nopad_wattention[i])))

#         hidden_sentence_encoding_out = Model(inputs=self.model.input,
#                                              outputs=self.model.get_layer('dense_transform_sentence').output)
#         hidden_sentence_encodings = np.squeeze(
#             hidden_sentence_encoding_out.predict(np.expand_dims(encoded_text, 0)), 0)
#         sentence_context = self.model.get_layer('sentence_attention').get_weights()[0]
#         u_sattention = np.exp(np.squeeze(np.dot(hidden_sentence_encodings, sentence_context), -1))
#         if websafe:
#             u_sattention = u_sattention.astype(float)
#         nopad_sattention = u_sattention[-len(normalized_text):]

#         nopad_sattention = nopad_sattention/np.expand_dims(np.sum(nopad_sattention, -1), -1)

#         activation_map = list(zip(word_activation_maps, nopad_sattention))

#         return activation_map


#     def load_weights(self, saved_model_dir, saved_model_filename):
#         with CustomObjectScope({'Attention': Attention}):
#             print(os.path.join(saved_model_dir, saved_model_filename))
#             self.model = load_model(os.path.join(saved_model_dir, saved_model_filename))
#             self.word_attention_model = self.model.get_layer('time_distributed_1').layer
#             tokenizer_path = os.path.join(
#                 saved_model_dir, self.get_tokenizer_filename(saved_model_filename))
#             tokenizer_state = pickle.load(open(tokenizer_path, "rb" ))
#             self.tokenizer = tokenizer_state['tokenizer']
#             self.MAX_SENTENCE_COUNT = tokenizer_state['maxSentenceCount']
#             self.MAX_SENTENCE_LENGTH = tokenizer_state['maxSentenceLength']
#             self.VOCABULARY_SIZE = tokenizer_state['vocabularySize']
#             self.create_reverse_word_index()